# Deep Deterministic Policy Gradient (DDPG)
---
This notebook implements a DDPG agent with Unity's Udacity-modified Reacher environment.

### 1. Import the Necessary Packages

In [1]:
#import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import time   

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below. 

Change the __*visible_environment*__ variable to enable training with (True) or without (False) a visible environment.

The output below indicates the utilization of a cpu or a gpu.

In [2]:
from unityagents import UnityEnvironment

# set these parameters
visible_environment = False 
one_agent = False # if false, then 20 agent environment

if visible_environment == True and one_agent == True:
    env = UnityEnvironment(file_name='../unity_environments/Reacher/Vis_one_agent/Reacher.x86_64')
elif visible_environment == False and one_agent == True:
    env = UnityEnvironment(file_name='../unity_environments/Reacher/NoVis_one_agent/Reacher.x86_64')
elif visible_environment == True and one_agent == False:
    env = UnityEnvironment(file_name='../unity_environments/Reacher/Vis_20_agents/Reacher.x86_64')
elif visible_environment == False and one_agent == False:
    env = UnityEnvironment(file_name='../unity_environments/Reacher/NoVis_20_agents/Reacher.x86_64')    
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)

# number of actions and states
action_size = brain.vector_action_space_size

states = env_info.vector_observations
state_size = states.shape[1]


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2a. Hyperparameter Search


In [3]:
random.seed(32)

BUFFER_SIZE = [10 ** random.uniform(a=6, b=6) for x in range(0, 100)]       # replay buffer size
BATCH_SIZE = [2** random.randint(a=7, b=7) for x in range(0, 100)]         # minibatch size
GAMMA = [random.uniform(a=0.99, b=0.99) for x in range(0, 100)]                 # discount factor
TAU = [10 ** random.uniform(a=-2, b=-2) for x in range(0, 100)]               # for soft update of target parameters
LR_ACTOR = [10 ** random.uniform(a=-3.1, b=-2.9) for x in range(0, 100)]             # learning rate of the actor 
LR_CRITIC = [3*(10 ** random.uniform(a=-4.1, b=-3.9)) for x in range(0, 100)]            # learning rate of the critic
WEIGHT_DECAY = np.zeros(100) # 0.0001   # L2 weight decay


from ddpg_agent import Agent

device =  cuda:0


### 3. Train the Agent with DDPG

Run the code cell below to train the agent from scratch.

Alternatively, **skip** to the next step below (**4. Watch a Smart Agent!**), to load the saved model weights from a pre-trained agent.

In [4]:
def plot_scores(scores, fig_indicator="normal"):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.ylim((-4, 24))
    plt.show()
    
    fig_name = "Results/Figure_" + fig_indicator + ".png"
    fig.savefig(fig_name)
    
    np.save("Results/scores_" + fig_indicator + ".npy", scores)
    

def ddpg(n_episodes=2000, max_t=700):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    
    tt = time.time()
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        agent.reset()
        score = np.zeros(num_agents)                          # initialize the score (for each agent)
        for t in range(max_t):
            
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]        # send the action to the environment

            next_states = env_info.vector_observations   # get the next state
            rewards = env_info.rewards                   # get the reward
            dones = env_info.local_done                  # see if episode has finished
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            score += rewards
            
            if np.any(dones):
                break 
        scores_deque.append(np.mean(score))
        scores.append(np.mean(score))
        
        if i_episode == 300:
            for param_group in agent.actor_optimizer.param_groups:
                param_group['lr'] = 1e-4 # 1e-3

            for param_group in agent.critic_optimizer.param_groups:
                param_group['lr'] = 1e-4 #3e-4    
        
            agent.TAU = 1e-2
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
        if i_episode % 100 == 0 or np.mean(scores_deque) >= 30.0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            elapsed = time.time() - tt # https://stackoverflow.com/questions/5849800/what-is-the-python-equivalent-of-matlabs-tic-and-toc-functions
            print('\t Elapsed Time: {:.2f}'.format(elapsed), 'seconds')
        if np.mean(scores_deque)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break 
        if (i_episode == 10 and np.mean(scores_deque) < 0.3):
            break
            
    return scores    

for ii in range(1, 2):
    print('ii = ', ii)

    
    BUFFER_SIZE = int(1e6)
    GAMMA = 0.99
    TAU = 1e-3
    WEIGHT_DECAY = 0
    for BATCH_SIZE in [32, 64, 128, 256, 512]:
        for LR_ACTOR in [1e-3, 3e-4, 1e-4]:
            for LR_CRITIC in [1e-3, 3e-4, 1e-4]:
                print(['buffer_size = ', int(BUFFER_SIZE), 'batch_size = ', int(BATCH_SIZE), 'gamma=', GAMMA, 
                   'tau=', TAU, 'lr_actor= ', LR_ACTOR, 'lr_critic=',LR_CRITIC, 'weight_decay=', WEIGHT_DECAY])
                t = time.time()
                agent = Agent(state_size=state_size, action_size=action_size, random_seed=0,
                              buffer_size=int(BUFFER_SIZE), batch_size=int(BATCH_SIZE), gamma=GAMMA, tau=TAU,
                               lr_actor=LR_ACTOR, lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY)
                scores = ddpg(n_episodes=300)
                elapsed = time.time() - t # https://stackoverflow.com/questions/5849800/what-is-the-python-equivalent-of-matlabs-tic-and-toc-functions
                print('\t Time to train network: {:.2f}'.format(elapsed), 'seconds')

#plot_scores(scores, fig_indicator="ddpg_normal_soln_final")

ii =  1
['buffer_size = ', 1000000, 'batch_size = ', 32, 'gamma=', 0.99, 'tau=', 0.001, 'lr_actor= ', 0.001, 'lr_critic=', 0.001, 'weight_decay=', 0]
Episode 1	Average Score: 0.95
Episode 2	Average Score: 0.76
Episode 3	Average Score: 0.75
Episode 4	Average Score: 0.74
Episode 5	Average Score: 0.76
Episode 6	Average Score: 0.77
Episode 7	Average Score: 0.77
Episode 8	Average Score: 0.76
Episode 9	Average Score: 0.74
Episode 10	Average Score: 0.73
Episode 11	Average Score: 0.72
Episode 12	Average Score: 0.72
Episode 13	Average Score: 0.71
Episode 14	Average Score: 0.70
Episode 15	Average Score: 0.70
Episode 16	Average Score: 0.69
Episode 17	Average Score: 0.70
Episode 18	Average Score: 0.71
Episode 19	Average Score: 0.70
Episode 20	Average Score: 0.70
Episode 21	Average Score: 0.70
Episode 22	Average Score: 0.70
Episode 23	Average Score: 0.71
Episode 24	Average Score: 0.71
Episode 25	Average Score: 0.71
Episode 26	Average Score: 0.71
Episode 27	Average Score: 0.70
Episode 28	Average Sco

Episode 252	Average Score: 0.71
Episode 253	Average Score: 0.71
Episode 254	Average Score: 0.71
Episode 255	Average Score: 0.70
Episode 256	Average Score: 0.70
Episode 257	Average Score: 0.70
Episode 258	Average Score: 0.70
Episode 259	Average Score: 0.71
Episode 260	Average Score: 0.70
Episode 261	Average Score: 0.71
Episode 262	Average Score: 0.71
Episode 263	Average Score: 0.71
Episode 264	Average Score: 0.71
Episode 265	Average Score: 0.71
Episode 266	Average Score: 0.71
Episode 267	Average Score: 0.71
Episode 268	Average Score: 0.71
Episode 269	Average Score: 0.71
Episode 270	Average Score: 0.71
Episode 271	Average Score: 0.71
Episode 272	Average Score: 0.71
Episode 273	Average Score: 0.71
Episode 274	Average Score: 0.71
Episode 275	Average Score: 0.71
Episode 276	Average Score: 0.71
Episode 277	Average Score: 0.71
Episode 278	Average Score: 0.71
Episode 279	Average Score: 0.71
Episode 280	Average Score: 0.71
Episode 281	Average Score: 0.71
Episode 282	Average Score: 0.71
Episode 

Episode 201	Average Score: 2.17
Episode 202	Average Score: 2.19
Episode 203	Average Score: 2.21
Episode 204	Average Score: 2.22
Episode 205	Average Score: 2.24
Episode 206	Average Score: 2.26
Episode 207	Average Score: 2.27
Episode 208	Average Score: 2.29
Episode 209	Average Score: 2.31
Episode 210	Average Score: 2.32
Episode 211	Average Score: 2.35
Episode 212	Average Score: 2.37
Episode 213	Average Score: 2.38
Episode 214	Average Score: 2.39
Episode 215	Average Score: 2.39
Episode 216	Average Score: 2.41
Episode 217	Average Score: 2.44
Episode 218	Average Score: 2.47
Episode 219	Average Score: 2.50
Episode 220	Average Score: 2.53
Episode 221	Average Score: 2.55
Episode 222	Average Score: 2.57
Episode 223	Average Score: 2.58
Episode 224	Average Score: 2.61
Episode 225	Average Score: 2.63
Episode 226	Average Score: 2.65
Episode 227	Average Score: 2.67
Episode 228	Average Score: 2.69
Episode 229	Average Score: 2.71
Episode 230	Average Score: 2.73
Episode 231	Average Score: 2.74
Episode 

Episode 151	Average Score: 0.26
Episode 152	Average Score: 0.26
Episode 153	Average Score: 0.26
Episode 154	Average Score: 0.26
Episode 155	Average Score: 0.26
Episode 156	Average Score: 0.26
Episode 157	Average Score: 0.26
Episode 158	Average Score: 0.26
Episode 159	Average Score: 0.26
Episode 160	Average Score: 0.26
Episode 161	Average Score: 0.26
Episode 162	Average Score: 0.26
Episode 163	Average Score: 0.26
Episode 164	Average Score: 0.26
Episode 165	Average Score: 0.26
Episode 166	Average Score: 0.26
Episode 167	Average Score: 0.26
Episode 168	Average Score: 0.26
Episode 169	Average Score: 0.26
Episode 170	Average Score: 0.26
Episode 171	Average Score: 0.26
Episode 172	Average Score: 0.26
Episode 173	Average Score: 0.26
Episode 174	Average Score: 0.26
Episode 175	Average Score: 0.26
Episode 176	Average Score: 0.26
Episode 177	Average Score: 0.26
Episode 178	Average Score: 0.26
Episode 179	Average Score: 0.26
Episode 180	Average Score: 0.26
Episode 181	Average Score: 0.26
Episode 

Episode 86	Average Score: 1.16
Episode 87	Average Score: 1.16
Episode 88	Average Score: 1.17
Episode 89	Average Score: 1.18
Episode 90	Average Score: 1.19
Episode 91	Average Score: 1.19
Episode 92	Average Score: 1.20
Episode 93	Average Score: 1.20
Episode 94	Average Score: 1.21
Episode 95	Average Score: 1.22
Episode 96	Average Score: 1.23
Episode 97	Average Score: 1.25
Episode 98	Average Score: 1.25
Episode 99	Average Score: 1.26
Episode 100	Average Score: 1.28
Episode 100	Average Score: 1.28
	 Elapsed Time: 553.38 seconds
Episode 101	Average Score: 1.29
Episode 102	Average Score: 1.32
Episode 103	Average Score: 1.33
Episode 104	Average Score: 1.35
Episode 105	Average Score: 1.38
Episode 106	Average Score: 1.39
Episode 107	Average Score: 1.41
Episode 108	Average Score: 1.44
Episode 109	Average Score: 1.46
Episode 110	Average Score: 1.48
Episode 111	Average Score: 1.50
Episode 112	Average Score: 1.52
Episode 113	Average Score: 1.53
Episode 114	Average Score: 1.55
Episode 115	Average Sco

...

Episode 183	Average Score: 8.26
Episode 184	Average Score: 8.28
Episode 185	Average Score: 8.29
Episode 186	Average Score: 8.29
Episode 187	Average Score: 8.30
Episode 188	Average Score: 8.32
Episode 189	Average Score: 8.33
Episode 190	Average Score: 8.35
Episode 191	Average Score: 8.37
Episode 192	Average Score: 8.37
Episode 193	Average Score: 8.39
Episode 194	Average Score: 8.38
Episode 195	Average Score: 8.39
Episode 196	Average Score: 8.40
Episode 197	Average Score: 8.40
Episode 198	Average Score: 8.43
Episode 199	Average Score: 8.44
Episode 200	Average Score: 8.46
Episode 200	Average Score: 8.46
	 Elapsed Time: 3997.44 seconds
Episode 201	Average Score: 8.47
Episode 202	Average Score: 8.48
Episode 203	Average Score: 8.51
Episode 204	Average Score: 8.53
Episode 205	Average Score: 8.52
Episode 206	Average Score: 8.53
Episode 207	Average Score: 8.54
Episode 208	Average Score: 8.54
Episode 209	Average Score: 8.52
Episode 210	Average Score: 8.52
Episode 211	Average Score: 8.51
Episode 

Episode 131	Average Score: 8.27
Episode 132	Average Score: 8.32
Episode 133	Average Score: 8.36
Episode 134	Average Score: 8.41
Episode 135	Average Score: 8.44
Episode 136	Average Score: 8.48
Episode 137	Average Score: 8.53
Episode 138	Average Score: 8.57
Episode 139	Average Score: 8.62
Episode 140	Average Score: 8.65
Episode 141	Average Score: 8.69
Episode 142	Average Score: 8.73
Episode 143	Average Score: 8.76
Episode 144	Average Score: 8.79
Episode 145	Average Score: 8.82
Episode 146	Average Score: 8.87
Episode 147	Average Score: 8.90
Episode 148	Average Score: 8.95
Episode 149	Average Score: 8.98
Episode 150	Average Score: 9.00
Episode 151	Average Score: 9.04
Episode 152	Average Score: 9.06
Episode 153	Average Score: 9.11
Episode 154	Average Score: 9.14
Episode 155	Average Score: 9.15
Episode 156	Average Score: 9.15
Episode 157	Average Score: 9.16
Episode 158	Average Score: 9.19
Episode 159	Average Score: 9.19
Episode 160	Average Score: 9.21
Episode 161	Average Score: 9.23
Episode 

Episode 80	Average Score: 3.08
Episode 81	Average Score: 3.13
Episode 82	Average Score: 3.18
Episode 83	Average Score: 3.24
Episode 84	Average Score: 3.26
Episode 85	Average Score: 3.31
Episode 86	Average Score: 3.34
Episode 87	Average Score: 3.37
Episode 88	Average Score: 3.40
Episode 89	Average Score: 3.43
Episode 90	Average Score: 3.47
Episode 91	Average Score: 3.51
Episode 92	Average Score: 3.54
Episode 93	Average Score: 3.58
Episode 94	Average Score: 3.62
Episode 95	Average Score: 3.66
Episode 96	Average Score: 3.71
Episode 97	Average Score: 3.75
Episode 98	Average Score: 3.78
Episode 99	Average Score: 3.82
Episode 100	Average Score: 3.87
Episode 100	Average Score: 3.87
	 Elapsed Time: 1718.25 seconds
Episode 101	Average Score: 3.94
Episode 102	Average Score: 4.01
Episode 103	Average Score: 4.08
Episode 104	Average Score: 4.16
Episode 105	Average Score: 4.24
Episode 106	Average Score: 4.33
Episode 107	Average Score: 4.43
Episode 108	Average Score: 4.53
Episode 109	Average Score: 4

Episode 23	Average Score: 0.85
Episode 24	Average Score: 0.91
Episode 25	Average Score: 0.95
Episode 26	Average Score: 0.96
Episode 27	Average Score: 0.99
Episode 28	Average Score: 1.05
Episode 29	Average Score: 1.05
Episode 30	Average Score: 1.05
Episode 31	Average Score: 1.06
Episode 32	Average Score: 1.11
Episode 33	Average Score: 1.15
Episode 34	Average Score: 1.18
Episode 35	Average Score: 1.21
Episode 36	Average Score: 1.26
Episode 37	Average Score: 1.30
Episode 38	Average Score: 1.34
Episode 39	Average Score: 1.38
Episode 40	Average Score: 1.42
Episode 41	Average Score: 1.48
Episode 42	Average Score: 1.53
Episode 43	Average Score: 1.58
Episode 44	Average Score: 1.64
Episode 45	Average Score: 1.69
Episode 46	Average Score: 1.74
Episode 47	Average Score: 1.80
Episode 48	Average Score: 1.86
Episode 49	Average Score: 1.91
Episode 50	Average Score: 1.95
Episode 51	Average Score: 1.99
Episode 52	Average Score: 2.05
Episode 53	Average Score: 2.09
Episode 54	Average Score: 2.15
Episode 

KeyboardInterrupt: 

### 4. Watch a Smart Agent!

The code below loads the trained weights from file to watch a smart agent. 

To visualize the trained environment,
 - change the __*visible_environment*__ variable to **True** in (**2. Instantiate the Environment and Agent**)
 - restart the kernel, and 
 - **skip** the previous section (**3. Train the Agent with DQN**).

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# load the weights from file
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

for i in range(1):
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations                  # get the current state (for each agent)
    for j in range(200):
        actions = agent.act(states)
        env_info = env.step(actions)[brain_name]        # send the action to the environment
        states = env_info.vector_observations
        dones = env_info.local_done                  # see if episode has finished
        
        if np.any(dones):
            break     

In [ ]:
env.close()